In [1]:
import requests
import json 
from dotenv import load_dotenv
import os
from pprint import pprint



In [2]:
load_dotenv()
key = os.getenv("RESCUE_API_KEY")
print(f'"{key}"')

"CyozmQMV"


Gathering breed information from RescueOrg API

In [3]:
headers = {
    'Content-Type': 'application/vnd.api+json',
    'Authorization': os.getenv('RESCUE_API_KEY')
}

response = requests.get('https://api.rescuegroups.org/v5/public/animals/species/', headers=headers)
data = response.json()
pprint(data)


{'data': [{'attributes': {'plural': 'Alpacas',
                          'singular': 'Alpaca',
                          'youngPlural': 'Crias',
                          'youngSingular': 'Cria'},
           'id': '1',
           'relationships': {'breeds': {'data': [{'id': '622',
                                                  'links': {'self': '/public/animals/species/1/breeds/622'},
                                                  'type': 'breeds'},
                                                 {'id': '651',
                                                  'links': {'self': '/public/animals/species/1/breeds/651'},
                                                  'type': 'breeds'}],
                                        'links': {'self': '/public/animals/species/1/breeds/'}},
                             'colors': {'data': [{'id': '348',
                                                  'links': {'self': '/public/animals/species/1/colors/348'},
                             

In [4]:
species = {}
for item in data['data']:
    species[item['attributes']['singular']] = item['id']
pprint(species)

{'Alpaca': '1',
 'Bird': '2',
 'Cat': '3',
 'Chicken': '4',
 'Chinchilla': '5',
 'Cow': '6',
 'Degu': '7',
 'Dog': '8',
 'Donkey': '9',
 'Duck': '10',
 'Ferret': '11',
 'Fish': '12',
 'Frog': '13',
 'Gecko': '14',
 'Gerbil': '15',
 'Goat': '16',
 'Goose': '17',
 'Groundhog': '18',
 'Guinea Pig': '19',
 'Hamster': '20',
 'Hedgehog': '21',
 'Hermit Crab': '22',
 'Horse': '23',
 'Iguana': '24',
 'Lizard': '25'}


This allowed us to get the proper species, dog, which is 8. Now to get breed IDS and make a dataframe

In [5]:
response = requests.get('https://api.rescuegroups.org/v5/public/animals/species/8/breeds/?limit=300', headers=headers)
data = response.json()
pprint(data)

{'data': [{'attributes': {'name': 'Affenpinscher'},
           'id': '74',
           'relationships': {'species': {'data': [{'id': '8',
                                                   'links': {'self': '/public/breeds/74/species/8'},
                                                   'type': 'species'}],
                                         'links': {'self': '/public/breeds/74/species/'}}},
           'type': 'breeds'},
          {'attributes': {'name': 'Afghan Hound'},
           'id': '75',
           'relationships': {'species': {'data': [{'id': '8',
                                                   'links': {'self': '/public/breeds/75/species/8'},
                                                   'type': 'species'}],
                                         'links': {'self': '/public/breeds/75/species/'}}},
           'type': 'breeds'},
          {'attributes': {'name': 'Airedale Terrier'},
           'id': '76',
           'relationships': {'species': {'data': [{'id': '8

In [6]:
breeds = [{'Name': item['attributes']['name'], 'ID': item['id']} for item in data['data']]
pprint(breeds)

[{'ID': '74', 'Name': 'Affenpinscher'},
 {'ID': '75', 'Name': 'Afghan Hound'},
 {'ID': '76', 'Name': 'Airedale Terrier'},
 {'ID': '77', 'Name': 'Akbash'},
 {'ID': '78', 'Name': 'Akita'},
 {'ID': '79', 'Name': 'Alaskan Malamute'},
 {'ID': '80', 'Name': 'American Bulldog'},
 {'ID': '81', 'Name': 'American Eskimo Dog'},
 {'ID': '82', 'Name': 'American Staffordshire Terrier'},
 {'ID': '83', 'Name': 'Anatolian Shepherd'},
 {'ID': '84', 'Name': 'Australian Cattle Dog/Blue Heeler'},
 {'ID': '85', 'Name': 'Australian Kelpie'},
 {'ID': '86', 'Name': 'Australian Shepherd'},
 {'ID': '87', 'Name': 'Australian Terrier'},
 {'ID': '88', 'Name': 'Basenji'},
 {'ID': '89', 'Name': 'Basset Hound'},
 {'ID': '90', 'Name': 'Beagle'},
 {'ID': '91', 'Name': 'Bearded Collie'},
 {'ID': '92', 'Name': 'Beauceron'},
 {'ID': '93', 'Name': 'Belgian Shepherd Dog Sheepdog'},
 {'ID': '94', 'Name': 'Bernese Mountain Dog'},
 {'ID': '95', 'Name': 'Bichon Frise'},
 {'ID': '96', 'Name': 'Black and Tan Coonhound'},
 {'ID': '

In [7]:
import pandas as pd
df = pd.DataFrame(breeds)
df['Name'] = df['Name'].str.lower()
df.head()

,Name,ID
0,affenpinscher,74
1,afghan hound,75
2,airedale terrier,76
3,akbash,77
4,akita,78


Save this table to use in compiling our breeds DB

In [29]:
df.to_csv('../data/breeds.csv', index=False)

Exploring how to get available animals by breed, location, and radius

In [6]:
params = {"data":{"filterProcessing":"1","filterRadius":{"miles":50,"postalCode":"90210"},"filters":[{"criteria":"145","fieldName":"animals.breedPrimaryId","operation":"equal"}]}}
out = json.dumps(params, indent=4)
print(out)

{
    "data": {
        "filterProcessing": "1",
        "filterRadius": {
            "miles": 50,
            "postalCode": "90210"
        },
        "filters": [
            {
                "criteria": "145",
                "fieldName": "animals.breedPrimaryId",
                "operation": "equal"
            }
        ]
    }
}


In [11]:
load_dotenv

headers = {
    'Content-Type': 'application/vnd.api+json',
    'Authorization': os.getenv('RESCUE_API_KEY')
}

data = {
    "data": {
        "filterProcessing": "1",
         "filterRadius": {
            "miles": 50,
            "postalcode": "90210"
        },
        "filters": [
            {
                "criteria": "145",
                "fieldName": "animals.breedPrimaryId",
                "operation": "equal",
                
            }
        ]
    }
}

url = 'https://api.rescuegroups.org/v5/public/animals/search/available'
response = requests.post(url, headers=headers, json=params)
print(response.status_code)
output = response.json()

400


In [12]:

pprint(output)


{'errors': [{'detail': 'You did not provide sufficient information to perform '
                       'a geolocation search',
             'source': {'data': 'filterRadius'},
             'status': 400,
             'title': 'Invalid Geolocation Filter'}],
 'meta': {'transactionId': 'rZmbQSMHjCQj'}}


In [58]:
pprint(output['included'][0:9])

[{'attributes': {'name': 'Golden Retriever'},
  'id': '145',
  'links': {'self': '/public/animals/breeds/145'},
  'type': 'breeds'},
 {'attributes': {'name': 'Chow Chow'},
  'id': '121',
  'links': {'self': '/public/animals/breeds/121'},
  'type': 'breeds'},
 {'attributes': {'name': 'Yellow'},
  'id': '58',
  'links': {'self': '/public/animals/colors/58'},
  'type': 'colors'},
 {'attributes': {'plural': 'Dogs',
                 'singular': 'Dog',
                 'youngPlural': 'Puppies',
                 'youngSingular': 'Puppy'},
  'id': '8',
  'links': {'self': '/public/animals/species/8'},
  'type': 'species'},
 {'attributes': {'description': 'Available for Adoption', 'name': 'Available'},
  'id': '1',
  'links': {'self': '/public/animals/statuses/1'},
  'type': 'statuses'},
 {'attributes': {'coordinates': '33.649, -117.918',
                 'lat': 33.649,
                 'lon': -117.918,
                 'phone': '(702) 249-3817'},
  'id': '1000003448',
  'type': 'locations'},
 

Organization offering a dog, by organization id from animal record

In [39]:
len(output['data'])

3

In [18]:
out = json.dumps(output['data'][0], indent=4)
print(out)

{
    "type": "animals",
    "id": "12121932",
    "attributes": {
        "distance": 81.58,
        "isAdoptionPending": false,
        "ageGroup": "Adult",
        "isBirthDateExact": false,
        "breedString": "Beagle / Labrador Retriever",
        "breedPrimary": "Beagle",
        "breedPrimaryId": 90,
        "breedSecondary": "Labrador Retriever",
        "breedSecondaryId": 162,
        "isCourtesyListing": false,
        "isCurrentVaccinations": true,
        "descriptionHtml": "<p>Spuds is a long legged lemon beagle/lab mix, 7 years old and 40-45&nbsp;lbs.&nbsp; He's got a stubbed tail - shelter thought he had some cocker spaniel in him - we don't see it. He has developed some great friendships with other B&amp;B dogs - might help if he was adopted with one of those companions.</p>\n<p><strong>Very selective of his people - not good with strangers. He will bite. But with the right people, he's a good boy- friendly, loves cuddles, mingles well with other dogs, runs around t

In [46]:
url = 'https://api.rescuegroups.org/v5/public/orgs/6685'
response = requests.get(url, headers=headers)
data = response.json()
pprint(data)


{'data': [{'attributes': {'about': 'Mission is No More Homeless Pets',
                          'adoptionProcess': 'http://bestfriends.org/Adopt-a-Pet/How-to-Adopt/',
                          'city': 'Kanab',
                          'citystate': 'Kanab, UT',
                          'coordinates': '37.2718, -111.8',
                          'country': 'United States',
                          'email': 'adoptions@bestfriends.org',
                          'facebookUrl': 'http://fb.com/bestfriendsanimalsociety',
                          'isCommonapplicationAccepted': False,
                          'lat': 37.27180099487305,
                          'lon': -111.80000305175781,
                          'name': 'Best Friends Animal Society',
                          'phone': '(435) 644-2001',
                          'postalcode': '84741',
                          'serveAreas': 'National',
                          'services': 'All',
                          'state': 'UT',
 

In [47]:
url = 'https://api.rescuegroups.org/v5/public/pictures/92101079'
response = requests.get(url, headers=headers)
data = response.json()
pprint(data)

{'errors': [{'detail': 'The endpoint URL you requested was not found',
             'source': {'path': ''},
             'status': 400,
             'title': 'Invalid endpoint'}],
 'meta': {'transactionId': 'RWBuVQdAmIJy'}}
